In [1]:
from drain.model import Drain,Node,LogCluster
from drain.optimizer import Optimizer
from drain.plotter import createPlot
from drain.partition import Partition
from utils.data_utils import *
import unsupervised.model as unsupervised_model
from torch import nn,optim
from torch.utils.data import  DataLoader

import pandas as pd
import os

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = 31
batch_size = 2048
input_size = 1
window_size = 10
num_layers = 2
hidden_size = 64

In [3]:
logPath = 'D:\\毕业设计\\loghub\\HDFS_1\\HDFS.log'
parsed_result = '.\\data\\parser\\log_item_to_label.csv'
cluster_result = '.\\data\\parser\\'
# partition
partition_output = '.\\data\\lstm\\dataset_official\\'
instance_file = 'instance.csv'
label_file = '.\\data\\partition\\anormaly_label.csv'
normal_output = 'normal.csv'
abnormal_output = 'abnormal.csv'

#lstm dataset
lstm_dataset = '.\\data\\lstm\\dataset_official\\'
unsupervised_train = 'train.csv'
unsupervised_test_normal = 'normal.csv'
unsupervised_test_abnormal = 'abnormal.csv'

predicted_input = 'unsupervised_input.csv'
predicted_normal_output = 'unsupervised_normal_output.csv'
predicted_abnormal_output = 'unsupervised_abnormal_output.csv'
supervised_test = 'supervised_test.csv'

In [4]:
def parse_log():

    rex = ['blk_(|-)[0-9]+', '(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)']
    removeCol = [0,1,2]
    myParser = Drain(rex=rex,removeCol=removeCol,st=0.4)
    myParser.fit(isReconstruct=True,inputFile=logPath,outputFile=parsed_result)

    myParser.save(savePath=cluster_result)

'''
partition phase
'''
def partition():

    partition = Partition(outputFileDir=partition_output,log_item_to_event_id_file=parsed_result,instances_file_path=instance_file,label_file_path=label_file,normal_output=normal_output,abnormal_output=abnormal_output)
    partition.partition_by_file()
    partition.map_log_seq_to_label()

    split_data(input_dir=partition_output,output_dir=lstm_dataset)

In [5]:
model_dir = 'model'
version = '0.1'
input_size = 1
model_usl_name = 'dev_v1.2_batch_size=2048_epoch=300'
# model_usl_name = 'type={}_version={}'.format('official_unsupervised',version)


In [6]:
model = unsupervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [12]:
if os.path.exists(model_dir + '/' + model_usl_name + '.pt'):
    model.load_state_dict(torch.load(model_dir + '/' + model_usl_name + '.pt'))
    print('loading model {} success.'.format(model_usl_name))

loading model dev_v1.2_batch_size=2048_epoch=300 success.


In [13]:
model

Model(
  (lstm): LSTM(1, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=31, bias=True)
)

In [8]:
lstm_dataset + unsupervised_train

'.\\data\\lstm\\dataset_official\\train.csv'

In [7]:
train_dataset = generate_train_data(lstm_dataset + unsupervised_train)
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

loading data: 100%|█████████████████████████████████████████████████████████████| 4855/4855 [00:00<00:00, 24651.26it/s]


Number of sessions(.\data\lstm\dataset_official\train.csv): 4855
Number of seqs(.\data\lstm\dataset_official\train.csv): 99980


In [21]:
train_dataset[20:40]

(tensor([[ 5., 11.,  9., 11.,  9., 11.,  9., 26., 26., 26.],
         [11.,  9., 11.,  9., 11.,  9., 26., 26., 26., 23.],
         [ 9., 11.,  9., 11.,  9., 26., 26., 26., 23., 23.],
         [11.,  9., 11.,  9., 26., 26., 26., 23., 23., 23.],
         [ 9., 11.,  9., 26., 26., 26., 23., 23., 23., 21.],
         [11.,  9., 26., 26., 26., 23., 23., 23., 21., 21.],
         [ 9., 26., 26., 26., 23., 23., 23., 21., 21., 21.],
         [ 0.,  5., 22.,  5.,  5., 11.,  9., 11.,  9., 11.],
         [ 5., 22.,  5.,  5., 11.,  9., 11.,  9., 11.,  9.],
         [22.,  5.,  5., 11.,  9., 11.,  9., 11.,  9., 26.],
         [ 5.,  5., 11.,  9., 11.,  9., 11.,  9., 26., 26.],
         [ 5., 11.,  9., 11.,  9., 11.,  9., 26., 26., 26.],
         [11.,  9., 11.,  9., 11.,  9., 26., 26., 26., 23.],
         [ 9., 11.,  9., 11.,  9., 26., 26., 26., 23., 23.],
         [11.,  9., 11.,  9., 26., 26., 26., 23., 23., 23.],
         [ 9., 11.,  9., 26., 26., 26., 23., 23., 23., 21.],
         [11.,  9., 26.,

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [9]:
model.train()
unsupervised_model.train(model, dataloader, criterion, optimizer, current_epoch=0, num_epochs=300)

RuntimeError: cuda runtime error (710) : device-side assert triggered at C:/w/1/s/tmp_conda_3.6_095855/conda/conda-bld/pytorch_1579082406639/work/aten/src\THC/generic/THCTensorMath.cu:26

In [86]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + model_usl_name + '.pt')
print('Finished Training')

Finished Training


In [13]:
model.eval()
batch_size = 20000
test_normal_session, test_normal_dataset, normal_block = generate_predicted_data(
    lstm_dataset + unsupervised_test_normal, window_size)
normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_abnormal_session, test_abnormal_dataset, abnormal_block = generate_predicted_data(
    lstm_dataset + unsupervised_test_abnormal, window_size)
abnormal_dataloader = DataLoader(test_abnormal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

.\data\lstm\dataset_official\normal.csv: 100%|████████████████████████████████| 14177/14177 [00:01<00:00, 12425.70it/s]


Number of sessions(.\data\lstm\dataset_official\normal.csv): 14177
Number of seqs(.\data\lstm\dataset_official\normal.csv): 269570


.\data\lstm\dataset_official\abnormal.csv: 100%|████████████████████████████████| 4123/4123 [00:00<00:00, 20713.52it/s]


Number of sessions(.\data\lstm\dataset_official\abnormal.csv): 4123
Number of seqs(.\data\lstm\dataset_official\abnormal.csv): 88410


In [ ]:
model.eval()
batch_size = 20000
test_normal_session, test_normal_dataset, normal_block = generate_predicted_data(
    lstm_dataset + 'no_dup_normal.csv', window_size)
normal_dataloader = DataLoader(test_normal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_abnormal_session, test_abnormal_dataset, abnormal_block = generate_predicted_data(
    lstm_dataset + 'no_dup_abnormal.csv', window_size)
abnormal_dataloader = DataLoader(test_abnormal_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [14]:
import unsupervised.predictor as unsupervised_predictor

In [16]:
re = unsupervised_predictor.predict(model, abnormal_dataloader,test_abnormal_session,10, window_size,ts=0.0005)

loading data: 5it [00:26,  5.33s/it]


In [15]:
test_normal_result, test_abnormal_result = unsupervised_predictor.count_metries(model, normal_dataloader, abnormal_dataloader,
                                                        test_normal_session, test_abnormal_session,
                                                        10, window_size,ts=0.0001)

loading data: 14it [01:10,  5.03s/it]
loading data: 5it [00:22,  4.52s/it]

elapsed_time: 93.172s
false positive (FP): 893, false negative (FN): 120, Precision: 81.761%, Recall: 97.089%, F1-measure: 88.768%
Finished Predicting


In [16]:
model.eval()
test_normal_result, test_abnormal_result = unsupervised_predictor.count_metries(model, normal_dataloader, abnormal_dataloader,
                                                        test_normal_session, test_abnormal_session,
                                                        10, window_size,ts=0)

loading data: 14it [01:09,  4.95s/it]
loading data: 5it [00:21,  4.37s/it]

elapsed_time: 91.204s
false positive (FP): 557, false negative (FN): 349, Precision: 87.139%, Recall: 91.535%, F1-measure: 89.283%
Finished Predicting


In [27]:
test_normal_result, test_abnormal_result = unsupervised_predictor.count_metries(model, normal_dataloader, abnormal_dataloader,
                                                        test_normal_session, test_abnormal_session,
                                                        10, window_size,ts=0.000)

loading data: 14it [01:22,  5.90s/it]
loading data: 5it [00:22,  4.47s/it]

elapsed_time: 105.094s
false positive (FP): 1114, false negative (FN): 369, Precision: 77.116%, Recall: 91.050%, F1-measure: 83.506%
Finished Predicting


In [18]:
predicted_input

'unsupervised_input.csv'

In [19]:
sessions, dataset, block_to_seqs = generate_predicted_data(lstm_dataset+predicted_input,window_size)

.\data\lstm\dataset_official\unsupervised_input.csv: 100%|███████████████████████| 4999/4999 [00:00<00:00, 5801.28it/s]


Number of sessions(.\data\lstm\dataset_official\unsupervised_input.csv): 4999
Number of seqs(.\data\lstm\dataset_official\unsupervised_input.csv): 93821


In [20]:
batch_size = 20000
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [21]:
normal_result = unsupervised_predictor.predict(model, dataloader,sessions,10, window_size,ts=0.0005)

loading data: 5it [00:24,  4.97s/it]


In [22]:
def show_result(result):
    TN = result[:3000].count(False)
    FN = result[3000:].count(False)
    print('TN:{} FN:{}'.format(TN,FN))
    print('Negative Precision:',TN/(TN+FN))
    TP = result[3000:].count(True)
    FP = result[:3000].count(True)
    print('TP:{} FP:{}'.format(TP,FP))
    print('Positive Precision:',TP/(TP+FP))

In [23]:
show_result(normal_result)

TN:2853 FN:31
Negative Precision: 0.989251040221914
TP:1968 FP:147
Positive Precision: 0.9304964539007092


In [24]:
abnormal_result = unsupervised_predictor.predict(model, dataloader,sessions,10, window_size,ts=0)

loading data: 5it [00:24,  4.82s/it]


In [68]:
show_result(abnormal_result)

TN:2938 FN:156
Negative Precision: 0.9495798319327731
TP:1843 FP:62
Positive Precision: 0.9674540682414698


In [39]:
f = open(lstm_dataset+predicted_normal_output,'w')
f.write('EventSequence\n')
for i,label in enumerate(normal_result):
    if label==False:
        f.write('"'+str(block_to_seqs[i][1])+'"\n')
f.close()

In [40]:
f = open(lstm_dataset+predicted_abnormal_output,'w')
f.write('EventSequence\n')
for i,label in enumerate(abnormal_result):
    if label:
        f.write('"'+str(block_to_seqs[i][1])+'"\n')
f.close()

In [6]:
input_size = 768

In [4]:
from utils.bert_utils import *

In [33]:
build_bert_cache('.\\data\\lstm\\dataset_official\\HDFS_templates.txt',"./data/lstm/bert_official_cache.pth")

In [36]:
predicted_normal_output

'unsupervised_normal_output.csv'

In [69]:
predicted_abnormal_output

'unsupervised_abnormal_output.csv'

In [5]:
train_dataset, test_dataset=generate_bert_data(lstm_dataset,predicted_normal_output,predicted_abnormal_output,"./data/lstm/bert_official_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 1895/1895 [00:43<00:00, 44.05it/s]


Number of sessions(.\data\lstm\dataset_official\): 4778
Number of normal sessions: 2883
Number of abnormal sessions: 1895


In [7]:
batch_size = 100
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [8]:
import supervised.model as supervised_model

In [9]:
model = supervised_model.Model(input_size, hidden_size, num_layers, num_classes).to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [13]:
model_sl_name = 'type={}_version={}'.format('official_supervised','v1')

In [51]:
if os.path.exists(model_dir + '/' + model_sl_name + '.pt'):
    print('loading model {} success.'.format(model_sl_name))
    model.load_state_dict(torch.load(model_dir + '/' + model_name + '.pt'))

In [52]:
torch.cuda.empty_cache()

In [14]:
model.train()
supervised_model.train(model,train_dataloader,criterion,optimizer,current_epoch=0,num_epochs=30,input_size=input_size)

Epoch [1/30], train_loss: 2.3085
Epoch [2/30], train_loss: 0.6736
Epoch [3/30], train_loss: 0.6589
Epoch [4/30], train_loss: 0.6476
Epoch [5/30], train_loss: 0.6379
Epoch [6/30], train_loss: 0.6371
Epoch [7/30], train_loss: 0.6232
Epoch [8/30], train_loss: 0.6073
Epoch [9/30], train_loss: 0.5388
Epoch [10/30], train_loss: 0.5965
Epoch [11/30], train_loss: 0.4843
Epoch [12/30], train_loss: 0.3699
Epoch [13/30], train_loss: 0.6199
Epoch [14/30], train_loss: 0.5685
Epoch [15/30], train_loss: 0.4054
Epoch [16/30], train_loss: 0.3473
Epoch [17/30], train_loss: 0.5635
Epoch [18/30], train_loss: 0.6030
Epoch [19/30], train_loss: 0.5710
Epoch [20/30], train_loss: 0.5686
Epoch [21/30], train_loss: 0.5673
Epoch [22/30], train_loss: 0.5660
Epoch [23/30], train_loss: 0.5618
Epoch [24/30], train_loss: 0.5560
Epoch [25/30], train_loss: 0.5156
Epoch [26/30], train_loss: 0.4590
Epoch [27/30], train_loss: 0.4268


KeyboardInterrupt: 

In [55]:
import numpy as np
def accuracy(y_pred, y_true):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    pred = np.argmax(y_pred.cpu().numpy(),1)
    for i,val in enumerate(pred):
        if val == 0:
            if y_true[i]==0:
                TN +=1
            else:
                FN +=1
        else:
            if y_true[i]==1:
                TP+=1
            else:
                FP+=1
    return TP,TN,FP,FN

In [56]:
def predict_result(model,data):
    model.eval()
    with torch.no_grad():
        epoch_loss = 0
        for step, (seq, label) in enumerate(data):
            seq = seq.clone().detach().view(-1, seq.shape[1], input_size).to(device)
            test_output = model(seq.to(device))
            if step == 0:
                output = test_output
                labels = label
            else:
                labels = torch.cat([labels, label], 0)
                output = torch.cat([output, test_output], 0)
            epoch_loss += criterion(test_output, label.to(device)).data
        TP,TN,FP,FN = accuracy(output, labels)
        epoch_accuracy = (TP+TN)/(FP+FN+TP+TN)
        epoch_loss = epoch_loss / len(data)
        print('TN:{} FN:{}'.format(TN,FN))
        print('Negative Precision:',round(TN/(TN+FN),3))
        print('TP:{} FP:{}'.format(TP,FP))
        print('Positive Precision:',round(TP/(TP+FP),3))
        print('loss: ', round(epoch_loss.item(), 3), 'accuracy: ', round(epoch_accuracy, 3))

In [70]:
predict_result(model,train_dataloader)

TN:1679 FN:33
Negative Precision: 0.981
TP:1108 FP:46
Positive Precision: 0.96
loss:  0.11 accuracy:  0.972


In [71]:
predict_result(model,test_dataloader)

TN:1137 FN:32
Negative Precision: 0.973
TP:722 FP:21
Positive Precision: 0.972
loss:  0.106 accuracy:  0.972


In [67]:
lstm_dataset

'.\\data\\lstm\\dataset_official\\'

In [59]:
supervised_input_normal = 'supervised_input_normal.csv'
supervised_input_abnormal = 'supervised_input_abnormal.csv'

In [82]:
test_dataset1,test_dataset2 =generate_bert_data(lstm_dataset,supervised_input_normal,supervised_input_abnormal,"./data/lstm/bert_official_cache.pth")

abnormal:: 100%|███████████████████████████████████████████████████████████████████| 2117/2117 [00:45<00:00, 46.64it/s]


Number of sessions(.\data\lstm\dataset_official\): 7111
Number of normal sessions: 4994
Number of abnormal sessions: 2117


In [76]:
train_dataset[:100][1]

tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 1])

In [81]:
torch.sum(test_dataset1[:][1])

tensor(1270)

In [83]:
test_dataloader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=True)
test_dataloader2 = DataLoader(test_dataset2, batch_size=batch_size)

In [84]:
predict_result(model,test_dataloader1)

TN:2790 FN:40
Negative Precision: 0.986
TP:1229 FP:207
Positive Precision: 0.856
loss:  0.196 accuracy:  0.942


In [85]:
predict_result(model,test_dataloader2)

TN:1898 FN:16
Negative Precision: 0.992
TP:832 FP:99
Positive Precision: 0.894
loss:  0.147 accuracy:  0.96


In [2]:
from unsupervised.wf_constructor import workflow_constructor,draw_wf


In [ ]:
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)
torch.save(model.state_dict(), model_dir + '/' + model_usl_name + '.pt')
print('Finished Training')

In [3]:
wf = workflow_constructor(model,input_size=1)
wf.generate_seq([0],1,3)


NameError: name 'model' is not defined

In [ ]:
wf.workflow_construction([0],{30})
draw_wf(wf)